In [2]:
pwd

'c:\\Users\\Dell\\Desktop\\Chicken\\Chicken-Disease-Classification\\research'

In [4]:
import os
os.chdir('../')

In [5]:
pwd


'c:\\Users\\Dell\\Desktop\\Chicken\\Chicken-Disease-Classification'

In [22]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class PrepareCallbacksConfig:
    root_dir : Path
    tensorboard_root_logs_dir : Path
    checkpoint_model_filepath : Path

In [23]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [24]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path= CONFIG_FILE_PATH,
            params_file_path= PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_logs_dir)
        ])

        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_logs_dir = Path(config.tensorboard_root_logs_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
        )

        return prepare_callbacks_config

In [25]:
import os
from urllib import request
from zipfile import ZipFile
import tensorflow as tf
import time

In [33]:
class PrepareCallbacks:
    def __init__(self, config : PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_logs_dir,
            f"tb_logs_at{timestamp}"
        )

        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return[
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [34]:
try:
    config = ConfigurationManager()
    prepare_callback_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallbacks(config=prepare_callback_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

except Exception as e:
    raise e


[2023-07-16 15:14:58,571: INFO: common: yaml file : c:\Users\Dell\Desktop\Chicken\Chicken-Disease-Classification\config\config.yaml is successfully added:]
[2023-07-16 15:14:58,579: INFO: common: yaml file : c:\Users\Dell\Desktop\Chicken\Chicken-Disease-Classification\params.yaml is successfully added:]
['artifacts']
[2023-07-16 15:14:58,579: INFO: common: directory : artifacts is successfully created:]
[WindowsPath('artifacts/prepare_callbacks/checkpoint_dir'), WindowsPath('artifacts/prepare_callbacks/tensorboard_logs_dir')]
[2023-07-16 15:14:58,584: INFO: common: directory : artifacts\prepare_callbacks\checkpoint_dir is successfully created:]
[2023-07-16 15:14:58,584: INFO: common: directory : artifacts\prepare_callbacks\tensorboard_logs_dir is successfully created:]
